In [23]:
import pandas as pd
import numpy as np
import zipfile
import os

from scipy.stats import entropy, skew
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report

In [24]:
zip_files = 'homework.zip'

with zipfile.ZipFile(zip_files, 'r') as zip_ref:
    zip_ref.extractall('unzip_folder')

In [25]:
def load_data_frame(folder_path):

    data = pd.DataFrame()
    type_activity = os.path.basename(folder_path)
    for filename in os.listdir(folder_path):

      if filename.endswith('.csv'):
        df = pd.read_csv(os.path.join(folder_path, filename))
        data_d = pd.DataFrame(
        {
        'max_x': df['accelerometer_X'].max(),
        'min_x': df['accelerometer_X'].min(),
        'mean_x': df['accelerometer_X'].mean(),
        'std_x': df['accelerometer_X'].std(),

        'max_y': df['accelerometer_Y'].max(),
        'min_y': df['accelerometer_Y'].min(),
        'mean_y': df['accelerometer_Y'].mean(),
        'std_y': df['accelerometer_Y'].std(),

        'max_z': df['accelerometer_Z'].max(),
        'min_z': df['accelerometer_Z'].min(),
        'mean_z': df['accelerometer_Z'].mean(),
        'std_z': df['accelerometer_Z'].std(),
        'type_activity': type_activity,
        }, index=[0]
        )
        data = pd.concat([data, data_d]).reset_index(drop=True)

    return data

In [26]:
walk_path = 'unzip_folder/data/walking'
stairs_path = 'unzip_folder/data/stairs'
run_path = 'unzip_folder/data/running'
idle_path = 'unzip_folder/data/idle'

walk_df = load_data_frame(walk_path)
stairs_df = load_data_frame(stairs_path)
run_df = load_data_frame(run_path)
idle_df = load_data_frame(idle_path)

In [27]:
df = pd.concat([walk_df, stairs_df, run_df, idle_df]).reset_index(drop=True)

In [28]:
df

,max_x,min_x,mean_x,std_x,max_y,min_y,mean_y,std_y,max_z,min_z,mean_z,std_z,type_activity
0,0.363919,-16.333244,-4.189055,4.385627,4.783615,-16.242264,-8.317297,5.038919,3.591303,-15.710752,-2.899857,4.888471,walking
1,16.132132,-7.043741,3.882118,4.434384,-3.907337,-19.378668,-11.663753,4.865778,15.710752,-19.038692,-0.581791,5.680558,walking
2,16.132132,-11.755530,1.832362,5.215808,-3.912125,-19.378668,-11.681470,4.858045,26.743233,-19.038692,-0.198878,8.125054,walking
3,9.375694,-12.876017,0.174936,5.623429,10.055647,-24.823084,-8.960220,7.006229,13.450625,-12.449849,-0.963107,5.741857,walking
4,4.156334,-11.951855,-3.197536,4.161091,9.251195,-17.894264,-7.924967,7.101022,24.076092,-22.108059,-1.519520,9.653732,walking
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6457,0.325611,0.220267,0.265916,0.025204,0.076614,-0.067038,-0.024421,0.028352,9.801862,9.730036,9.765630,0.017903,idle
6458,0.828394,-0.885855,-0.082999,0.388219,7.465121,6.090849,6.741593,0.303691,9.260772,5.975928,7.064013,0.643167,idle
6459,0.488417,-0.861913,-0.240218,0.308297,7.139510,4.563349,5.941770,0.608575,10.237606,5.702989,7.539182,0.838954,idle
6460,2.035071,-0.459687,0.067516,0.518483,7.580043,3.840300,4.576277,0.757588,9.892841,3.423709,8.333897,1.181953,idle


In [29]:
X = df.drop('type_activity', axis=1)
y = df['type_activity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [30]:
# Random Forest
model_rf = RandomForestClassifier(n_estimators=100)
model_rf.fit(X_train, y_train)
predictions_rf = model_rf.predict(X_test)
print("Random Forest:")
print(classification_report(y_test, predictions_rf))

# SVM
model_svm = svm.SVC()
model_svm.fit(X_train, y_train)
predictions_svm = model_svm.predict(X_test)
print("SVM:")
print(classification_report(y_test, predictions_svm))

Random Forest:
              precision    recall  f1-score   support

        idle       1.00      1.00      1.00       190
     running       1.00      1.00      1.00       677
      stairs       1.00      0.94      0.97        35
     walking       0.99      1.00      1.00       391

    accuracy                           1.00      1293
   macro avg       1.00      0.99      0.99      1293
weighted avg       1.00      1.00      1.00      1293
SVM:
              precision    recall  f1-score   support

        idle       1.00      1.00      1.00       190
     running       1.00      1.00      1.00       677
      stairs       0.95      0.57      0.71        35
     walking       0.96      1.00      0.98       391

    accuracy                           0.99      1293
   macro avg       0.98      0.89      0.92      1293
weighted avg       0.99      0.99      0.99      1293


In [31]:
# Обидві моделі добре підходять для цього типу задачі. RandomForest показала себе краще, ніж SVM для цього набору даних. Вона демонструє відмінну точність, повноту і F1-score 1.0 для всіх класів.

In [32]:
# Random Forest
param_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

random_rf = RandomizedSearchCV(RandomForestClassifier(), param_rf, n_iter=10, cv=5)
random_rf.fit(X_train, y_train)

print("Best parameters for Random Forest: ", random_rf.best_params_)

# SVM
param_svm = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid']
}

random_svm = RandomizedSearchCV(svm.SVC(), param_svm, n_iter=10, cv=5)
random_svm.fit(X_train, y_train)

print("Best parameters for SVM: ", random_svm.best_params_)

Best parameters for Random Forest:  {'n_estimators': 50, 'min_samples_split': 2, 'max_depth': 30}
Best parameters for SVM:  {'kernel': 'poly', 'gamma': 1, 'C': 1}


In [33]:
# RandomizedSearchCV працює дуже схоже до GridSearchCV, але замість того, щоб пробувати всі можливі комбінації параметрів, він випадковим чином вибирає певну кількість комбінацій. Це може бути набагато швидше, особливо якщо у вас багато параметрів для налаштування.